# Compass Analysis

Here is a bunch of text - I hope it looks normal! (make stuff hidden)

In [1]:
# Required for notebook path to be at head of project for torch_fairness imports
import sys
sys.path.insert(0, os.path.abspath('../../..'))
import os
os.chdir('../../..')

In [2]:
import os

import pandas as pd

from torch_fairness.util import SensitiveGroup, infer_minority_majority, dummy_code_sensitive

## Prepare dataset

In [3]:
filename = os.path.join("datasets", "compas-scores.csv")
sensitive_cols = ['sex', 'race', 'age_cat']
label_col = 'two_year_recid'
features_cols = ["juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree"]

In [4]:
data = pd.read_csv(filename)
# https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
data = data[(data['days_b_screening_arrest'] <= 30) 
             & (data['days_b_screening_arrest'] >= -30) 
             & (data['is_recid'] != -1) 
             & (data['c_charge_degree'] != "O") 
             & (data['score_text'] != "N/A")]
data = data[sensitive_cols+[label_col] + features_cols]
data.head()

,sex,race,age_cat,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree
0,Male,Other,Greater than 45,0,0,0,0,0,F
1,Male,African-American,25 - 45,1,0,0,0,0,F
2,Male,African-American,Less than 25,1,0,0,1,4,F
5,Male,Other,25 - 45,0,0,0,0,0,M
6,Male,Caucasian,25 - 45,1,0,0,0,14,F


In [5]:
sensitive_map = infer_minority_majority(data, item_names=sensitive_cols)
print(sensitive_map)

[SensitiveGroup(majority='Male', minority=['Female'], variable_name='sex'), SensitiveGroup(majority='African-American', minority=['Caucasian', 'Hispanic', 'Other', 'Asian'], variable_name='race'), SensitiveGroup(majority='25 - 45', minority=['Less than 25', 'Greater than 45'], variable_name='age_cat')]


In [7]:
sensitive = dummy_code_sensitive(data[sensitive_cols], sensitive_map=sensitive_map)
sensitive.head()

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
data['c_charge_degree'].unique()